# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.212143     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.270509     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [11]:
model_all.coefficients.print_rows(18,4)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
import numpy as np
np.logspace(1, 7, num=13)

array([1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07])

In [22]:
rss = []
for i,l1 in enumerate(np.logspace(1, 7, num=13)):
    model = turicreate.linear_regression.create(training, target = 'price', features = all_features,
                                                validation_set = validation, 
                                                l2_penalty = 0., l1_penalty = l1)
    pred = model.predict(validation)
    rs = pred - validation['price']
    rss.append(sum(rs**2))
print(rss)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 10        | 11       | 0.000003  | 0.298462     | 5334292.166784     | 4616363.170973       | 273638.544684                   | 254871.319748                     |

+-----------+----------+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

[625766285142461.2, 625766285362395.4, 625766286057886.9, 625766288257224.8, 625766295212185.9, 625766317206077.6, 625766386760661.5, 625766606749281.1, 625767302791633.1, 625769507643885.0, 625776517727025.6, 625799062845466.8, 625883719085424.4]


In [25]:
rss = np.array(rss)
l1s = np.logspace(1, 7, num=13)

In [26]:
l1s[rss.argmin()]

10.0

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [27]:
model = turicreate.linear_regression.create(training, target = 'price', features = all_features,
                                                validation_set = validation, 
                                                l2_penalty = 0., l1_penalty = 10)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.213795     | 5737572.318059     | 5148098.475109       | 289374.784200                   | 269493.457699                     |

| 10        | 11       | 0.000003  | 0.299955     | 5329815.359960     | 4609435.483176       | 273608.586502                   | 254847.407999                     |

+-----------+----------+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [31]:
model.coefficients.print_rows(18,4)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [32]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [37]:
nzero = []
for l1 in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target = 'price', features = all_features,
                                                validation_set = None, 
                                                l2_penalty = 0., l1_penalty = l1)
    nzero.append(model.coefficients['value'].nnz())

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000012  | 0.175111     | 7239807.218168     | 388380.843122                   |

| 10        | 11       | 0.000012  | 0.233088     | 7135517.863156     | 380130.779816                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [45]:
print(l1_penalty_values)
print(nzero)

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]
[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [46]:
nzero.index(10)

14

In [44]:
nzero.index(6)

15

In [41]:
l1_penalty_min = l1_penalty_values[nzero.index(10)]
l1_penalty_max = l1_penalty_values[nzero.index(6)]
print (l1_penalty_min, l1_penalty_max)

2976351441.6313133 3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [49]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [52]:
def RSS(model):
    prediction = model.predict(validation)
    residual = validation['price'] - prediction
    residual_squared = residual * residual
    RSS = residual_squared.sum()
    return RSS

for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    nonzero_weights = model.coefficients['value'].nnz()
    print (l1_penalty)
    print ('nonzero weights number: ' + str(nonzero_weights))
    print ('RSS: ' + str(RSS(model)))

2976351441.6313133
nonzero weights number: 10
RSS: 966925692362085.2
3019316638.9524155
nonzero weights number: 10
RSS: 974019450084554.6
3062281836.2735176
nonzero weights number: 10
RSS: 981188367942453.8
3105247033.5946198
nonzero weights number: 10
RSS: 989328342459473.5
3148212230.915722
nonzero weights number: 10
RSS: 998783211265890.1
3191177428.236824
nonzero weights number: 10
RSS: 1008477167020094.4
3234142625.557926
nonzero weights number: 10
RSS: 1018298780553819.4
3277107822.8790283
nonzero weights number: 10
RSS: 1028247992205977.0
3320073020.2001305
nonzero weights number: 8
RSS: 1034616909232827.4
3363038217.5212326
nonzero weights number: 8
RSS: 1038554735941040.2
3406003414.8423347
nonzero weights number: 8
RSS: 1043237237871701.2
3448968612.163437
nonzero weights number: 7
RSS: 1046937488751711.5
3491933809.484539
nonzero weights number: 7
RSS: 1051147625612862.5
3534899006.805641
nonzero weights number: 7
RSS: 1055992735342999.4
3577864204.126743
nonzero weights num

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [55]:
l1_penalty = np.linspace(l1_penalty_min,l1_penalty_max,20)[11]
best_model2 = turicreate.linear_regression.create(training, target='price', features=all_features,
                                               validation_set=None, verbose = False,
                                               l2_penalty=0., l1_penalty=l1_penalty)
feature_list = best_model2.coefficients['name']
coeff_list = best_model2.coefficients['value']
#print best_model2['coefficients']['value'].nnz()
index = 0
for coeff in coeff_list:
    if coeff != 0.0:
        print (feature_list[index])
    index += 1

(intercept)
bedrooms
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above
